In [1]:
import numpy as np

import pandas as pd
import time
import matplotlib.pyplot as plt
import re, string, timeit
from nltk.tag.stanford import StanfordNERTagger
import pickle
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
from scipy.sparse import hstack
pd.set_option('mode.chained_assignment',None)
import itertools

import random

In [2]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC
from textblob import TextBlob
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
df = pd.read_pickle('data/final_Processed_df.pkl')

In [4]:
df.head()

,authors,date,headline,link,short_description,NERtext,text,category
0,Melissa Jeltsen,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,There Were CARDINAL Mass Shootings In GPE DATE...,There Were 2 Mass Shootings In Texas Last Week...,CRIME
1,Andy McDonald,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,Will Smith Joins Diplo And PERSON For WORK_OF_...,Will Smith Joins Diplo And Nicky Jam For The 2...,ENTERTAINMENT
2,Ron Dicker,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,PERSON For The ORDINAL Time At DATE The actor ...,Hugh Grant Marries For The First Time At Age 5...,ENTERTAINMENT
3,Ron Dicker,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,PERSON Blasts GPE PERSON And NORP In New Artwo...,Jim Carrey Blasts Castrato Adam Schiff And Dem...,ENTERTAINMENT
4,Ron Dicker,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",PERSON Uses PERSON Poop Bags To Pick Up After ...,Julianna Margulies Uses Donald Trump Poop Bags...,ENTERTAINMENT


In [5]:
for i in df['text'].head():
    print(i)

There Were 2 Mass Shootings In Texas Last Week But Only 1 On TV She left her husband He killed their children Just another day in America
Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup 's Official Song Of course it has a song
Hugh Grant Marries For The First Time At Age 57 The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony
Jim Carrey Blasts Castrato Adam Schiff And Democrats In New Artwork The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump
Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog The Dietland actress said using the bags is a really cathartic therapeutic moment


In [6]:

for i in df['NERtext'].head():
    print(i)

There Were CARDINAL Mass Shootings In GPE DATE But DATE On TV She left her husband He killed their children Just another day in GPE
Will Smith Joins Diplo And PERSON For WORK_OF_ART Of course it has a song
PERSON For The ORDINAL Time At DATE The actor and his longtime girlfriend PERSON tied the knot in a civil ceremony
PERSON Blasts GPE PERSON And NORP In New Artwork The actor gives NORP an ass-kicking for not fighting hard enough against PERSON
PERSON Uses PERSON Poop Bags To Pick Up After Her Dog The NORP actress said using the bags is a really cathartic therapeutic moment


In [7]:
###X_train, X_test, y_train, y_test  =train_test_split(df['text'], df['category'], test_size=0.2, random_state=1, stratify=df['category'])
#X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1, stratify=y_train)

In [19]:
class model():
    def __init__(self, df, textColumn = 'text', SEED = 2):
        random.seed(SEED)
        self.SEED = SEED
        self.df = df

        #text column is whether we use NERtext or normal text column
        self.textColumn = textColumn
        self.le = LabelEncoder()
        self.df['category_code'] = self.le.fit_transform(df['category'])

        
    def lem(self, phrase, customized__stoppedWords):
        if customized__stoppedWords:
            stop_words_ = ['a', 'an', 'the', 'for', 'in', 'it', 'its', 'on', 'at', 'they', 'them',
                  'their', 'theirs', 'that', 'what', 'which', 'has', 'have', 'had',
                  'having', 'thus', 'do', 'no', 'nor', 'not', 'is', 'are', 'was',
                  'were', 'be', 'been', 'being', 'did', 'in', 'out', 'both', 'each',
                  'few', 'he', 'she', 'him', 'her', 'these', 'those', 'but', 'if',
                  'of', 'after', 'before', 'up', 'down', 'over', 'under',
                  'again', 'then', 'there', 'some', 'such', 'other',
                  'only', 'same', 'so', 'than', 'and', 'to']
        else:
            stop_words_ = list(set(stopwords.words('english')))
        wnl=WordNetLemmatizer()
        lemmed=[]
        for i in word_tokenize(phrase):
            w1=wnl.lemmatize(i, 'v')
            w2=wnl.lemmatize(w1, 'a')
            w3=wnl.lemmatize(w2, 'n')
            if i not in (stop_words_):
                lemmed.append(w3.lower())
        return " ".join(lemmed)


        #return " ".join([wnl.lemmatize(words.lower(), pos="v") for words in word_tokenize(phrase) if (words not in (stop_words_))])
    def add_sentiment(self):
        Polarity=[]
        Subjectivity=[]
        for i in range(len(self.df[self.textColumn])):
            x = TextBlob(" ".join(word_tokenize(self.df[self.textColumn][i]))).sentiment
            Polarity.append(x[0])
            Subjectivity.append(x[1])
        self.df['Polarity'] = Polarity
        self.df['Subjectivity'] = Subjectivity
    def add_wordCount(self):
        self.df['countLen']  = [len(word_tokenize(i)) for i in df[self.textColumn]]

    def fit(self, addAuthor = False, customized__stoppedWords= False, add_sentiment = False, add_wordLen = False):
        print('-----------Sample Text---------')
        for i in self.df[self.textColumn].head(3):
            print(i)
        self.df[self.textColumn] = self.df[self.textColumn].apply(lambda x: self.lem(x, customized__stoppedWords))


        if addAuthor:
            self.df[self.textColumn] = self.df[self.textColumn] +' ' + self.df['authors']


        print('-----------Sample Text---------')
        for i in self.df[self.textColumn].head(3):
            print(i)
        
        #self.add_sentiment()
        #self.add_wordCount()
        '''
        self.X_train, self.X_test, self.y_train, self.y_test  =train_test_split(self.df[self.textColumn], self.df['category_code'], test_size=0.2, random_state=self.SEED, stratify=self.df['category_code'])
        self.X_train, self.X_val, self.y_train, self.y_val  = train_test_split(self.X_train, self.y_train, test_size=0.25, random_state=1, stratify=self.y_train)
        
        
        vect_word = TfidfVectorizer(max_features=60000, lowercase=False, analyzer='word',tokenizer=word_tokenize,ngram_range=(1,3),dtype=np.float32)
        self.tr_vect = vect_word.fit_transform(self.X_train)
        self.vl_vect = vect_word.transform(self.X_val)
        
        print(self.tr_vect.shape[1] , 'total Width of the train vector' )
        if add_sentiment: 
            self.tr_vect = hstack((self.tr_vect, self.X_train[['Polarity','Subjectivity']].values),format='csr')
            self.vl_vect = hstack((self.vl_vect, self.X_val[['Polarity','Subjectivity']].values),format='csr')
        if add_wordLen: 
            self.tr_vect = hstack((self.tr_vect, self.X_train[['countLen']].values),format='csr')
            self.vl_vect = hstack((self.vl_vect, self.X_val[['countLen']].values),format='csr')
        print(self.tr_vect.shape[1] , 'total Width of the train vector' )
        
        
        #test_vect = vect_word.transform(X_test)
        
        







        #logistic regression
        print('Fitting LGR-------')
        t = time.process_time()
        self.lgr = LogisticRegression(solver='saga',multi_class='multinomial', max_iter=1000, C=1,verbose = True, n_jobs = -1, random_state = self.SEED).fit(self.tr_vect, self.y_train)
        self.lgr_y_pred = self.lgr.predict(self.vl_vect)
        elapsed_time = time.process_time() - t
        print('total elappsed time: ', elapsed_time)
        print('LGR accuracy: ', (self.lgr_y_pred == self.y_val).sum()/len(self.y_val))
        
        #print(classification_report(np.asarray(self.le.inverse_transform(self.y_val)),np.asarray( self.le.inverse_transform(self.lgr_y_pred))))


        #linear SVC
        t = time.process_time()
        print('Fitting LinearSVC-------')
        self.svc = LinearSVC(verbose = True, random_state = self.SEED).fit(self.tr_vect, self.y_train)
        self.svc_y_pred = self.svc.predict(self.vl_vect)
        print('total elappsed time: ', elapsed_time)
        print('LinearSVC: ', (self.svc_y_pred == self.y_val).sum()/len(self.y_val))

        #print(classification_report(np.asarray(self.le.inverse_transform(self.y_val)),np.asarray( self.le.inverse_transform(self.svc_y_pred))))
        '''

In [10]:
SEED = 2
textColumnToUse = 'NERtext'
addAuthor = True
nermodel = model(df, textColumnToUse, addAuthor, SEED)
nermodel.fit()

-----------Sample Text---------
there be cardinal mass shoot in gpe date but date on tv she leave husband he kill children just another day gpe melissa jeltsen
will smith join diplo and person for work_of_art of course song andy mcdonald
person for the ordinal time at date the actor longtime girlfriend person tie knot civil ceremony ron dicker


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 22 epochs took 34 seconds
LGR accuracy: 

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   34.0s finished


 0.7082721366159668
[LibLinear]LinearSVC:  0.7256229618381419


In [11]:
textColumnToUse = 'text'
model = model(df, textColumnToUse, addAuthor, SEED)
model.fit()

-----------Sample Text---------
there be 2 mass shoot in texas last week but only 1 on tv she leave husband he kill children just another day america melissa jeltsen
will smith join diplo and nicky jam for the 2018 world cup 's official song of course song andy mcdonald
hugh grant marry for the first time at age 57 the actor longtime girlfriend anna eberstein tie knot civil ceremony ron dicker


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 19 epochs took 29 seconds
LGR accuracy:  0.7315974210251176


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   29.1s finished


[LibLinear]LinearSVC:  0.7522839859600209


In [19]:
SEED = 2
textColumnToUse = 'text'
addAuthor = True
nermodel = model(df, textColumnToUse, addAuthor, SEED)
nermodel.fit(customized__stoppedWords= True)

-----------Sample Text---------
2 mass shoot texas last week 1 tv leave husband kill child just another day america melissa jeltsen Melissa Jeltsen
will smith join diplo nicky jam 2018 world cup 's official song course song andy mcdonald Andy McDonald
hugh grant marry first time age 57 actor longtime girlfriend anna eberstein tie knot civil ceremony ron dicker Ron Dicker


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 21 epochs took 29 seconds
LGR accuracy:  0.7399118767269921


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   28.7s finished


[LibLinear]LinearSVC:  0.755993129371935


In [14]:
wnl=WordNetLemmatizer()
lemmed=[]
for i in word_tokenize(df['text'][0]+' ' + df['authors'][0]):
    w1=wnl.lemmatize(i, 'v')
    w2=wnl.lemmatize(w1, 'a')
    w3=wnl.lemmatize(w2, 'n')
    
    lemmed.append(w3.lower())
    print('lemmed', w3)
print(" ".join(lemmed))


lemmed There
lemmed Were
lemmed 2
lemmed Mass
lemmed Shootings
lemmed In
lemmed Texas
lemmed Last
lemmed Week
lemmed But
lemmed Only
lemmed 1
lemmed On
lemmed TV
lemmed She
lemmed leave
lemmed her
lemmed husband
lemmed He
lemmed kill
lemmed their
lemmed child
lemmed Just
lemmed another
lemmed day
lemmed in
lemmed America
lemmed Melissa
lemmed Jeltsen
there were 2 mass shootings in texas last week but only 1 on tv she leave her husband he kill their child just another day in america melissa jeltsen


In [18]:
SEED = 2
textColumnToUse = 'text'
addAuthor = True
model1 = model(df.head(5), textColumnToUse, addAuthor, SEED)
model1.fit(customized__stoppedWords= True)

-----------Sample Text---------
there were 2 mass shootings in texas last week but only 1 on tv she leave husband he kill child just another day america melissa jeltsen Melissa Jeltsen
will smith joins diplo and nicky jam for the 2018 world cup 's official song of course song andy mcdonald Andy McDonald
hugh grant marries for the first time at age 57 the actor his longtime girlfriend anna eberstein tie knot civil ceremony ron dicker Ron Dicker
-----------Sample Text---------
2 mass shoot texas last week 1 tv leave husband kill child just another day america melissa jeltsen melissa jeltsen
will smith join diplo nicky jam 2018 world cup 's official song course song andy mcdonald andy mcdonald
hugh grant marry first time age 57 actor his longtime girlfriend anna eberstein tie knot civil ceremony ron dicker ron dicker


In [41]:
report = classification_report(np.asarray(model1.le.inverse_transform(model1.y_val)),np.asarray( model1.le.inverse_transform(model1.svc_y_pred)), output_dict=True)
report_df = pd.DataFrame(report)

In [45]:
report_df.transpose().to_csv('data/report.csv')

In [47]:
SEED = 2
textColumnToUse = 'NERtext'
addAuthor = True
nermodel = model(df, textColumnToUse, addAuthor, SEED)
nermodel.fit(customized__stoppedWords= True)

-----------Sample Text---------
There Were CARDINAL Mass Shootings In GPE DATE But DATE On TV She leave husband He kill child Just another day GPE Melissa Jeltsen Melissa Jeltsen
Will Smith Joins Diplo And PERSON For WORK_OF_ART Of course song Andy McDonald Andy McDonald
PERSON For The ORDINAL Time At DATE The actor his longtime girlfriend PERSON tie knot civil ceremony Ron Dicker Ron Dicker
60000 total Width of the train vector
60000 total Width of the train vector
Fitting LGR-------


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 21 epochs took 29 seconds
total elappsed time:  27.953125
LGR accuracy:  0.7070772447785716


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   28.6s finished


                precision    recall  f1-score   support

          ARTS       0.75      0.42      0.54       302
ARTS & CULTURE       0.65      0.54      0.59       268
  BLACK VOICES       0.73      0.47      0.57       905
      BUSINESS       0.61      0.57      0.59      1188
       COLLEGE       0.72      0.40      0.52       229
        COMEDY       0.61      0.58      0.60      1035
         CRIME       0.62      0.54      0.58       681
CULTURE & ARTS       0.93      0.32      0.48       206
       DIVORCE       0.89      0.68      0.78       685
     EDUCATION       0.75      0.35      0.48       201
 ENTERTAINMENT       0.69      0.81      0.74      3212
   ENVIRONMENT       0.88      0.29      0.44       264
         FIFTY       0.81      0.44      0.57       280
  FOOD & DRINK       0.86      0.80      0.83      1245
     GOOD NEWS       0.78      0.47      0.58       279
         GREEN       0.60      0.35      0.44       524
HEALTHY LIVING       0.61      0.68      0.64  

In [19]:
#Test with subjectivity len and polarity
'''SEED = 2
textColumnToUse = 'text'
addAuthor = True
nermodel = model(df, textColumnToUse, addAuthor, SEED)
nermodel.fit(customized__stoppedWords= True,add_sentiment = True, add_wordLen = True)'''

"SEED = 2\ntextColumnToUse = 'text'\naddAuthor = True\nnermodel = model(df, textColumnToUse, addAuthor, SEED)\nnermodel.fit(customized__stoppedWords= True,add_sentiment = True, add_wordLen = True)"

In [20]:
#Test with subjectivity len and polarity
'''SEED = 2
textColumnToUse = 'NERtext'
addAuthor = True
nermodel = model(df, textColumnToUse, addAuthor, SEED)
nermodel.fit(customized__stoppedWords= True,add_sentiment = True, add_wordLen = True)'''

"SEED = 2\ntextColumnToUse = 'NERtext'\naddAuthor = True\nnermodel = model(df, textColumnToUse, addAuthor, SEED)\nnermodel.fit(customized__stoppedWords= True,add_sentiment = True, add_wordLen = True)"

In [ ]:
# best model features exam the responses

model1.predit(model1.)



In [82]:
model1.svc = LinearSVC(verbose = True).fit(model1.tr_vect, model1.y_train)
model1.svc_y_pred = model1.svc.predict(model1.vl_vect)

[LibLinear]

In [83]:
print('LinearSVC: ', (model1.svc_y_pred == model1.y_val).sum()/len(model1.y_val))

LinearSVC:  0.6924896069303726


In [65]:
SEED = 2
textColumnToUse = 'NERtext'
addAuthor = True
nermodel = model(df, textColumnToUse, addAuthor, SEED)
nermodel.fit(customized__stoppedWords= True,add_sentiment = True, add_wordLen = True)

[26, 24, 26, 28, 28]

In [53]:
##################################################Results######################################
# without author NER LGR accuracy:  0.5542804510716686 LinearSVC:  0.554429812551343, BERT accuracy: 0.6484048366546631
# without author LGR accuracy:  0.6009310198899703 LinearSVC:  0.6112618555674492 BERT accuracy: 0.6991676092147827

# with Author NER LGR accuracy:  0.7082721366159668 LinearSVC:   0.7256229618381419, BERT: 0.7828295230865479
# with Author LGR accuracy:  0.7315974210251176 LinearSVC:  0.7522839859600209, BERT: 0.8054925203323364

#---------tailored stop words--------------------
# with Author NER LGR aacurac 0.6973687485997361, LinearSVC:  0.7229095616240572
# with Author LGR aacuracy: 0.7399118767269921 LinearSVC:  0.755993129371935



In [12]:
df.groupby(by='category').size().sort_values(ascending = False)

category
POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3651
WOMEN              3490
IMPACT             3459
DIVORCE            3426
CRIME              3405
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
WORLDPOST          2579
RELIGION           2556
STYLE              2254
SCIENCE            2178
WORLD NEWS         2177
TASTE              2096
TECH               2082
MONEY              1707
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
ARTS & CULTURE     1339
ENVIRONMENT        1323
COLLEGE            1144
LATINO VOICES      1129
CULTURE & ARTS     1030
EDUCATION          1004
dtype: 

In [46]:
test_features = ['TASTE', 'WEDDINGS']
test_df = df[df['category'].isin(test_features) ]

In [47]:
for i in test_df.head()['text']:
    print(i)

It 's Weird That American Restaurants Serve Ice Water In Winter But why do we even have ice in our drinks in the first place
Pineapple Casserole The Southern Dish That 's A Paradox Of Flavors It 's got pineapple cheddar and a whole lot of butter
How To Actually Get A Bartender 's Attention Plus other things they wish you knew
Diet Coke 's Millennial-Inspired Makeover Leaves People Befuddled It 's not like a regular soda it 's a cool soda
We Tested The New Tearless Onions To See If They Really Work Put away your goggles people


In [48]:
for i in test_df.head()['NERtext']:
    print(i)

It 's Weird That American Restaurants Serve Ice Water In Winter But why do we even have ice in our drinks in the ORDINAL place
Pineapple Casserole The Southern Dish That 's A Paradox Of Flavors It 's got pineapple cheddar and a whole lot of butter
How To Actually Get A Bartender 's Attention Plus other things they wish you knew
PERSON Millennial - Inspired Makeover Leaves People Befuddled It 's not like a regular soda it 's a cool soda
We Tested The New Tearless Onions To See If They Really Work Put away your goggles people


In [49]:
SEED = 2
textColumnToUse = 'NERtext'
addAuthor = True
nermodel = model(test_df, textColumnToUse, addAuthor, SEED)
nermodel.fit()

-----------Sample Text---------
it 's weird that american restaurants serve ice water in winter but even ice drink ordinal place todd van luling
pineapple casserole the southern dish that 's a paradox of flavor it 's get pineapple cheddar whole lot butter kristen aiken
how to actually get a bartender 's attention plus things wish know taylor pittman


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


convergence after 20 epochs took 0 seconds
LGR accuracy:  0.9834782608695652
[LibLinear]LinearSVC:  0.9904347826086957


In [50]:
SEED = 2
textColumnToUse = 'text'
addAuthor = True
nermodel = model(test_df, textColumnToUse, addAuthor, SEED)
nermodel.fit()

-----------Sample Text---------
it 's weird that american restaurants serve ice water in winter but even ice drink first place todd van luling
pineapple casserole the southern dish that 's a paradox of flavor it 's get pineapple cheddar whole lot butter kristen aiken
how to actually get a bartender 's attention plus things wish know taylor pittman


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


convergence after 21 epochs took 0 seconds
LGR accuracy:  0.9904347826086957
[LibLinear]LinearSVC:  0.9947826086956522
